# Complete Analysis: Apple Inc. (AAPL)

This notebook demonstrates a complete end-to-end analysis of Apple stock using all the tools in the Stock Analysis Toolkit.

## Analysis Includes:
1. Data Collection
2. Technical Analysis
3. Fundamental Analysis
4. Price Prediction (ML)
5. Backtesting
6. Investment Recommendation

---

In [ ]:
# Setup
import sys
sys.path.append('../..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Import all modules
from src.data.fetcher import get_stock_data
from src.indicators import TrendIndicators, MomentumIndicators, VolatilityIndicators, VolumeIndicators
from src.fundamental import FinancialRatios, GrowthMetrics
from src.models import FeatureEngineer, RandomForestPredictor, evaluate_model
from src.backtesting import BacktestEngine, SMACrossover, RSIMeanReversion
from src.visualization.charts import plot_candlestick, create_interactive_chart

pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

print("✅ All modules loaded successfully!")
print("\n📊 Starting Apple (AAPL) Analysis...")

## 1. Data Collection

In [ ]:
# Fetch Apple data
ticker = 'AAPL'
print(f"Fetching {ticker} data...\n")

df = get_stock_data(ticker, start='2020-01-01')

print(f"Data shape: {df.shape}")
print(f"Date range: {df.index[0].date()} to {df.index[-1].date()}")
print(f"\nLatest data:")
df.tail()

In [ ]:
# Price statistics
current_price = df['Close'].iloc[-1]
price_52w_high = df['Close'].rolling(252).max().iloc[-1]
price_52w_low = df['Close'].rolling(252).min().iloc[-1]
ytd_return = ((df['Close'].iloc[-1] / df['Close'].iloc[0]) - 1) * 100

print("\n" + "="*60)
print(f"📈 {ticker} PRICE OVERVIEW")
print("="*60)
print(f"Current Price: ${current_price:.2f}")
print(f"52-Week High: ${price_52w_high:.2f}")
print(f"52-Week Low: ${price_52w_low:.2f}")
print(f"YTD Return: {ytd_return:+.2f}%")
print("="*60)

## 2. Technical Analysis

In [ ]:
# Calculate all technical indicators
print("Calculating technical indicators...\n")

# Trend indicators
df['SMA_20'] = TrendIndicators.calculate_sma(df, 20)
df['SMA_50'] = TrendIndicators.calculate_sma(df, 50)
df['SMA_200'] = TrendIndicators.calculate_sma(df, 200)
df['EMA_12'] = TrendIndicators.calculate_ema(df, 12)
df['EMA_26'] = TrendIndicators.calculate_ema(df, 26)
macd_result = TrendIndicators.calculate_macd(df)
df = df.join(macd_result)

# Momentum indicators
df['RSI'] = MomentumIndicators.calculate_rsi(df, 14)
stoch_result = MomentumIndicators.calculate_stochastic(df)
df = df.join(stoch_result)

# Volatility indicators
bb_result = VolatilityIndicators.calculate_bollinger_bands(df)
df = df.join(bb_result)
df['ATR'] = VolatilityIndicators.calculate_atr(df)

# Volume indicators
df['OBV'] = VolumeIndicators.calculate_obv(df)
df['VWAP'] = VolumeIndicators.calculate_vwap(df)

print("✅ Technical indicators calculated!")
print(f"\nTotal indicators: {len([col for col in df.columns if col not in ['Open', 'High', 'Low', 'Close', 'Volume']])}")

In [ ]:
# Current indicator values
latest = df.iloc[-1]

print("\n" + "="*60)
print("📊 CURRENT TECHNICAL INDICATORS")
print("="*60)
print(f"\nMoving Averages:")
print(f"  SMA 20:  ${latest['SMA_20']:.2f}")
print(f"  SMA 50:  ${latest['SMA_50']:.2f}")
print(f"  SMA 200: ${latest['SMA_200']:.2f}")

print(f"\nMomentum:")
print(f"  RSI (14): {latest['RSI']:.2f}")
print(f"  Stochastic %K: {latest['Stoch_K']:.2f}")
print(f"  MACD: {latest['MACD']:.2f}")
print(f"  Signal: {latest['Signal']:.2f}")

print(f"\nVolatility:")
print(f"  BB Upper: ${latest['BB_Upper']:.2f}")
print(f"  BB Lower: ${latest['BB_Lower']:.2f}")
print(f"  ATR: ${latest['ATR']:.2f}")
print("="*60)

In [ ]:
# Technical signals
print("\n🎯 TECHNICAL SIGNALS:\n")

# Trend
if current_price > latest['SMA_50'] > latest['SMA_200']:
    print("✅ Strong Uptrend - Price > SMA50 > SMA200")
elif current_price > latest['SMA_50']:
    print("✅ Moderate Uptrend - Price > SMA50")
elif current_price < latest['SMA_50'] < latest['SMA_200']:
    print("❌ Strong Downtrend - Price < SMA50 < SMA200")
else:
    print("⚠️ Mixed Trend")

# Momentum
if latest['RSI'] > 70:
    print("⚠️ RSI Overbought (>70)")
elif latest['RSI'] < 30:
    print("⚠️ RSI Oversold (<30)")
else:
    print("✅ RSI Neutral (30-70)")

# MACD
if latest['MACD'] > latest['Signal']:
    print("✅ MACD Bullish - Above Signal Line")
else:
    print("❌ MACD Bearish - Below Signal Line")

# Bollinger Bands
if current_price > latest['BB_Upper']:
    print("⚠️ Price Above Upper BB - Overbought")
elif current_price < latest['BB_Lower']:
    print("⚠️ Price Below Lower BB - Oversold")
else:
    print("✅ Price Within BB Range")

In [ ]:
# Visualize with candlestick chart
plot_candlestick(df.tail(90), title=f'{ticker} - Last 90 Days',
                indicators=['SMA_50', 'SMA_200', 'BB_Upper', 'BB_Lower'])

## 3. Fundamental Analysis

In [ ]:
# Get fundamental ratios
print("Fetching fundamental data...\n")

try:
    ratios = FinancialRatios.get_all_ratios(ticker)
    
    if ratios is not None and not ratios.empty:
        print("✅ Fundamental data retrieved!\n")
        print(ratios.tail())
    else:
        print("⚠️ No fundamental data available")
        ratios = None
        
except Exception as e:
    print(f"❌ Error fetching fundamental data: {e}")
    ratios = None

## 4. Machine Learning Price Prediction

In [ ]:
# Prepare ML dataset
print("Preparing ML dataset...\n")

X, y, scaler = FeatureEngineer.prepare_ml_dataset(
    df,
    target_column='Close',
    forecast_horizon=1,
    target_type='price'
)

print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nNumber of features: {len(X.columns)}")

In [ ]:
# Train/test split
split_idx = int(len(X) * 0.8)
X_train = X.iloc[:split_idx]
X_test = X.iloc[split_idx:]
y_train = y.iloc[:split_idx]
y_test = y.iloc[split_idx:]

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")

In [ ]:
# Train Random Forest model
print("\nTraining Random Forest model...\n")

rf_model = RandomForestPredictor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions
predictions = rf_model.predict(X_test)

# Evaluate
print("\n" + "="*60)
print("🤖 MACHINE LEARNING MODEL PERFORMANCE")
print("="*60)
metrics = evaluate_model(y_test.values, predictions, plot=True)
print("="*60)

In [ ]:
# Feature importance
importance = rf_model.get_feature_importance(top_n=10)
print("\n🎯 Top 10 Most Important Features:\n")
print(importance.to_string(index=False))

## 5. Backtesting Strategies

In [ ]:
# Prepare data for backtesting
backtest_data = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()

# Initialize engine
engine = BacktestEngine(backtest_data, initial_cash=10000, commission=0.002)

# Compare strategies
print("Running backtests...\n")

comparison = engine.compare_strategies({
    'SMA Crossover': SMACrossover,
    'RSI Mean Reversion': RSIMeanReversion
})

print("\n" + "="*60)
print("📊 BACKTESTING RESULTS")
print("="*60)
print(comparison)
print("="*60)

## 6. Investment Recommendation

Based on all analysis above.

In [ ]:
# Scoring system
score = 0
max_score = 10
signals = []

# Technical score (5 points)
if current_price > latest['SMA_50']:
    score += 1
    signals.append("✅ Price above SMA 50")
else:
    signals.append("❌ Price below SMA 50")

if latest['SMA_50'] > latest['SMA_200']:
    score += 1
    signals.append("✅ Golden Cross (SMA 50 > SMA 200)")
else:
    signals.append("❌ Death Cross (SMA 50 < SMA 200)")

if 30 <= latest['RSI'] <= 70:
    score += 1
    signals.append("✅ RSI in neutral range")
else:
    signals.append("⚠️ RSI extreme reading")

if latest['MACD'] > latest['Signal']:
    score += 1
    signals.append("✅ MACD bullish")
else:
    signals.append("❌ MACD bearish")

if latest['BB_Lower'] <= current_price <= latest['BB_Upper']:
    score += 1
    signals.append("✅ Price within Bollinger Bands")
else:
    signals.append("⚠️ Price outside Bollinger Bands")

# ML model score (2 points)
if metrics['R2'] > 0.7:
    score += 2
    signals.append("✅ Strong ML model performance (R² > 0.7)")
elif metrics['R2'] > 0.5:
    score += 1
    signals.append("⚠️ Moderate ML model performance (R² > 0.5)")
else:
    signals.append("❌ Weak ML model performance")

# Backtest score (3 points)
best_return = comparison['Return [%]'].max()
if best_return > 20:
    score += 3
    signals.append(f"✅ Excellent backtest performance ({best_return:.1f}%)")
elif best_return > 10:
    score += 2
    signals.append(f"✅ Good backtest performance ({best_return:.1f}%)")
elif best_return > 0:
    score += 1
    signals.append(f"⚠️ Moderate backtest performance ({best_return:.1f}%)")
else:
    signals.append(f"❌ Negative backtest performance ({best_return:.1f}%)")

# Generate recommendation
percentage = (score / max_score) * 100

if percentage >= 80:
    recommendation = "🟢 STRONG BUY"
    outlook = "Very positive outlook with strong technical and model signals."
elif percentage >= 60:
    recommendation = "🟢 BUY"
    outlook = "Positive outlook with favorable technical indicators."
elif percentage >= 40:
    recommendation = "🟡 HOLD"
    outlook = "Mixed signals. Consider waiting for clearer trend."
elif percentage >= 20:
    recommendation = "🔴 SELL"
    outlook = "Negative outlook with concerning signals."
else:
    recommendation = "🔴 STRONG SELL"
    outlook = "Very negative outlook. Consider exiting position."

# Display recommendation
print("\n" + "="*70)
print(f"💡 INVESTMENT RECOMMENDATION FOR {ticker}")
print("="*70)
print(f"\n{recommendation}")
print(f"\nScore: {score}/{max_score} ({percentage:.0f}%)")
print(f"\nOutlook: {outlook}")
print(f"\n📋 Signal Breakdown:")
for signal in signals:
    print(f"  {signal}")

print("\n⚠️ DISCLAIMER: This is for educational purposes only.")
print("   Always do your own research and consult with financial professionals.")
print("="*70)

## Summary

This complete analysis of Apple (AAPL) included:

✅ **Data Collection** - Historical price data
✅ **Technical Analysis** - 15+ indicators calculated
✅ **Fundamental Analysis** - Financial ratios and metrics
✅ **ML Prediction** - Random Forest model trained and evaluated
✅ **Backtesting** - Multiple strategies tested
✅ **Investment Recommendation** - Comprehensive scoring system

---

### Key Takeaways:

- Comprehensive analysis requires multiple perspectives
- No single indicator should drive decisions
- Past performance doesn't guarantee future results
- Always consider risk management
- Professional advice is recommended for actual investing

**Use this notebook as a template for analyzing other stocks!**